In [1]:
#imports
import pandas as pd
import datetime

In [2]:
#read in csvs
demographics = pd.read_csv('Data/player_demographics.csv')
position_player = pd.read_csv('Data/position_player.csv')
hall_of_fame = pd.read_csv('Data/hall_of_fame.csv')
pitchers = pd.read_csv('Data/pitchers.csv')
appearances = pd.read_csv('Data/appearances.csv')

In [3]:
appearances = appearances.fillna(0, axis=1)

In [4]:
position_player.shape

(19428, 19)

In [5]:
pitchers.shape

(9655, 14)

In [6]:
#pitchers cleaning and merging 
pitchers_df = pd.merge(pitchers, demographics, on='playerID', how='left')
pitchers_hall_df = pd.merge(pitchers_df, hall_of_fame, on='playerID', how='left')
pitchers_complete = pitchers_hall_df[['playerID', 'nameFirst_x',
       'nameLast_x','Seasons', 'Wins', 'Loses', 'Games', 'Games_Started',
       'Complete_Games', 'Shut_Outs', 'Earned_Runs', 'Strike_Outs', 'Walks',
       'Games_Finished', 'Runs_Allowed', 'Batters_Faced',  'weight', 'height', 
       'bats', 'throws', 'finalGame','yearid', 'inducted']]
pitchers_complete = pitchers_complete.rename(columns={'nameFirst_x':'nameFirst', 'nameLast_x': 'nameLast'})
pitchers_complete['yearid'].fillna(0, inplace=True)
pitchers_complete['inducted'].fillna('N',inplace=True)
pitchers_complete.head()

,playerID,nameFirst,nameLast,Seasons,Wins,Loses,Games,Games_Started,Complete_Games,Shut_Outs,...,Games_Finished,Runs_Allowed,Batters_Faced,weight,height,bats,throws,finalGame,yearid,inducted
0,aardsda01,David,Aardsma,9,16,18,331,0,0,0,...,141,169,1475.0,215.0,75.0,R,R,2015-08-23 00:00:00,0.0,N
1,aasedo01,Don,Aase,13,66,60,448,91,22,5,...,235,503,4730.0,190.0,75.0,R,R,1990-10-03 00:00:00,0.0,N
2,abadfe01,Fernando,Abad,9,8,27,363,6,0,0,...,96,137,1350.0,220.0,73.0,L,L,2017-10-01 00:00:00,0.0,N
3,abbeybe01,Bert,Abbey,6,22,40,79,65,52,0,...,14,442,2568.0,175.0,71.0,R,R,1896-09-23 00:00:00,0.0,N
4,abbeych01,Charlie,Abbey,1,0,0,1,0,0,0,...,1,3,12.0,169.0,68.0,L,L,1897-08-19 00:00:00,0.0,N


In [7]:
#position players cleaning and merging
position_player_df = pd.merge(position_player, demographics, on='playerID', how='left')
position_player_hall_df = pd.merge(position_player_df, hall_of_fame, on='playerID', how='left')
position_player_edited = position_player_hall_df[['playerID', 'nameFirst_x', 'nameLast_x',
       'Seasons', 'Hits', 'RBIs', 'Runs', 'Stolen_Bases',
       'Strike_Outs', 'Home_Runs', 'Walks', 'Games', 'Triples', 'Doubles',
       'Caught_Stealing', 'At_Bats', 'Intentional_Walks', 'Hit_By_Pitch',
       'Sacrifice_Hits', 'Sacrifice_Flies', 'Ground_Into_Double_Plays', 'weight', 'height', 'bats', 'throws',
       'finalGame','yearid', 'inducted']]
position_player_edited = position_player_edited.rename(columns={'nameFirst_x':'nameFirst', 
                                                                    'nameLast_x':'nameLast'})
position_player_complete = pd.merge(position_player_edited, appearances, on='playerID', how='left')
position_player_complete = position_player_complete.fillna(0)
position_player_complete['primary_position'] = position_player_complete[['Games_Pitched', 'Games_Catcher', 
       'Games_First_Base','Games_Second_Base', 'Games_Third_Base', 'Games_Short_Stop',
       'Games_Left_Field', 'Games_Right_Field', 'Games_Center_Field',
       'Games_Out_Field', 'Games_DH']].idxmax(axis=1)
position_player_complete.head()

,playerID,nameFirst,nameLast,Seasons,Hits,RBIs,Runs,Stolen_Bases,Strike_Outs,Home_Runs,...,Games_First_Base,Games_Second_Base,Games_Third_Base,Games_Short_Stop,Games_Left_Field,Games_Right_Field,Games_Center_Field,Games_Out_Field,Games_DH,primary_position
0,aardsda01,David,Aardsma,9,0,0.0,0,0.0,2.0,0,...,0,0,0,0,0,0,0,0,0.0,Games_Pitched
1,aaronha01,Hank,Aaron,23,3771,2297.0,2174,240.0,1383.0,755,...,210,43,7,0,315,2174,308,2760,201.0,Games_Out_Field
2,aaronto01,Tommie,Aaron,7,216,94.0,102,9.0,145.0,13,...,232,7,10,0,135,2,1,137,0.0,Games_First_Base
3,aasedo01,Don,Aase,13,0,0.0,0,0.0,3.0,0,...,0,0,0,0,0,0,0,0,0.0,Games_Pitched
4,abadan01,Andy,Abad,3,2,0.0,1,0.0,5.0,0,...,8,0,0,0,0,1,0,1,0.0,Games_First_Base


In [8]:
#function to fix primary positions and format it 
def position_strip(position):
    position = position.split('_')[1:]
    if len(position)>1:
        return '_'.join(position)
    else:
        return position.pop(0)
#apply position player strip function
position_player_complete['primary_position']=position_player_complete['primary_position'].map(position_strip)

In [9]:
# remove pitchers from position player table
position_player_no_pitch = position_player_complete[position_player_complete['primary_position']!='Pitched']

In [10]:
#Remove unecessary columns from position player dataframe
position_player_no_pitch = position_player_no_pitch[['playerID', 'nameFirst', 'nameLast', 
       'Seasons', 'Hits', 'RBIs', 'Runs','Stolen_Bases', 'Strike_Outs', 'Home_Runs', 'Walks', 'Games', 'Triples',
       'Doubles', 'Caught_Stealing', 'At_Bats', 'Intentional_Walks','Hit_By_Pitch', 'Sacrifice_Hits', 
       'Sacrifice_Flies','Ground_Into_Double_Plays', 'weight', 'height', 'bats', 'throws',
       'finalGame', 'yearid', 'inducted', 'Total_Games', 'Games_Started','primary_position']]

In [11]:
position_player_no_pitch.columns

Index(['playerID', 'nameFirst', 'nameLast', 'Seasons', 'Hits', 'RBIs', 'Runs',
       'Stolen_Bases', 'Strike_Outs', 'Home_Runs', 'Walks', 'Games', 'Triples',
       'Doubles', 'Caught_Stealing', 'At_Bats', 'Intentional_Walks',
       'Hit_By_Pitch', 'Sacrifice_Hits', 'Sacrifice_Flies',
       'Ground_Into_Double_Plays', 'weight', 'height', 'bats', 'throws',
       'finalGame', 'yearid', 'inducted', 'Total_Games', 'Games_Started',
       'primary_position'],
      dtype='object')

Code to make pitchers data include eligibility for hall of fame

In [12]:
#function to simplify dates
def date_strip(value):
    return value.split(' ')[0]
date_strip('1970-05-03 00:00:00')

'1970-05-03'

In [13]:
pitchers_complete['finalGame'] = pitchers_complete['finalGame'].astype('str').map(date_strip)
pitchers_complete.finalGame = pd.to_datetime(pitchers_complete['finalGame'], format='%Y-%m-%d')
pitchers_complete['years_since_final_game']=(pd.datetime.today()-pitchers_complete['finalGame']).dt.days/365

In [14]:
# hall eligibility function
def eligible_for_hall(row):
    if (row['Seasons']>=10) & (row['years_since_final_game']>=5):
        return 'eligible'
    else:
        return 'not_eligible'

In [15]:
pitchers_complete['eligible_for_hall'] = pitchers_complete.apply(eligible_for_hall, axis=1)


Code to make position players data include columns with hall of fame eligibility

In [16]:
position_player_no_pitch['finalGame'] = position_player_no_pitch['finalGame'].astype('str').map(date_strip)
position_player_final = position_player_no_pitch[position_player_no_pitch['finalGame']!='0']
position_player_final.finalGame = pd.to_datetime(position_player_final['finalGame'], format='%Y-%m-%d')
position_player_final['years_since_final_game']=(pd.datetime.today()-position_player_final['finalGame']).dt.days/365

/Users/chris.marchetti/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/Users/chris.marchetti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [17]:
position_player_final['eligible_for_hall'] = position_player_final.apply(eligible_for_hall, axis=1)
position_player_final.inducted.replace(0, 'N', inplace=True)
position_player_final['primary_position'].replace(
    ['Left_Field','Right_Field', 'Center_Field'], 'Out_Field', inplace=True)
position_player_final.primary_position.value_counts()

/Users/chris.marchetti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/chris.marchetti/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Out_Field      3785
Catcher        1858
Second_Base    1211
Third_Base     1126
Short_Stop     1121
First_Base     1041
DH              106
Name: primary_position, dtype: int64

In [18]:
# Deal with position players who have pitched
playerID_list = position_player_final.playerID.tolist()
pitchers_final = pitchers_complete[~pitchers_complete['playerID'].isin(playerID_list)]
pitchers_final.head()

,playerID,nameFirst,nameLast,Seasons,Wins,Loses,Games,Games_Started,Complete_Games,Shut_Outs,...,Batters_Faced,weight,height,bats,throws,finalGame,yearid,inducted,years_since_final_game,eligible_for_hall
0,aardsda01,David,Aardsma,9,16,18,331,0,0,0,...,1475.0,215.0,75.0,R,R,2015-08-23,0.0,N,4.224658,not_eligible
1,aasedo01,Don,Aase,13,66,60,448,91,22,5,...,4730.0,190.0,75.0,R,R,1990-10-03,0.0,N,29.128767,eligible
2,abadfe01,Fernando,Abad,9,8,27,363,6,0,0,...,1350.0,220.0,73.0,L,L,2017-10-01,0.0,N,2.115068,not_eligible
3,abbeybe01,Bert,Abbey,6,22,40,79,65,52,0,...,2568.0,175.0,71.0,R,R,1896-09-23,0.0,N,123.216438,not_eligible
5,abbotda01,Dan,Abbott,1,0,2,3,1,1,0,...,67.0,190.0,71.0,R,R,1890-05-23,0.0,N,129.558904,not_eligible


In [19]:
# write out csv's for position players and pitchers

pitchers_final.to_csv('Data/pitchers_complete.csv', index=False)
position_player_final.to_csv('Data/position_players_complete.csv', index=False)